In [1]:
import math
import re
from dataclasses import dataclass, asdict
from itertools import combinations
from typing import Tuple, Type, TypeVar
from collections import defaultdict
from functools import lru_cache

import numpy as np

In [2]:
searched_sum = 2020

In [3]:
%%timeit
with open("day_01_input.txt") as input_data:
    for i, pair in enumerate(combinations(sorted((int(x) for x in input_data)), 2)):
        if sum(pair) == searched_sum:
            day_01_p1 = math.prod(pair)
            break

228 µs ± 31.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit
with open("day_01_input.txt") as input_data:
    for i, triple in enumerate(combinations(sorted((int(x) for x in input_data)), 3)):
        if sum(triple) == searched_sum:
            day_01_p2 = math.prod(triple)
            break

In [ ]:
parse_expression = re.compile(r"^(\d+)-(\d+)\s+(\w):\s+(.*)$")

def parse_entry(entry: str) -> Tuple:
    g = parse_expression.match(entry).groups()
    return *map(int, g[:2]), *g[2:]

In [ ]:
%%timeit
with open("day_02_input.txt") as input_data:
    day_02_p1 = sum(f <= p.count(c) <= s for f, s, c, p in map(parse_entry, input_data))

In [ ]:
%%timeit
with open("day_02_input.txt") as input_data:
    day_02_p2 = sum(
        (p[f - 1] == c) != (p[s - 1] == c)
        for f, s, c, p in map(parse_entry, input_data)
    )

In [ ]:
slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]

In [ ]:
%%timeit
with open("day_03_input.txt") as input_data:
    tree_map = np.array(
        [np.array([c == "#" for c in row.strip()], dtype=bool) for row in input_data]
    )
height, width = tree_map.shape

day_03_p1 = np.sum(tree_map[(np.arange(height), np.arange(0, height * 3, 3) % width)])

In [ ]:
%%timeit
with open("day_03_input.txt") as input_data:
    tree_map = np.array(
        [np.array([c == "#" for c in row.strip()], dtype=bool) for row in input_data]
    )
height, width = tree_map.shape

slope_index_generator = (
    (np.arange(0, height, d), np.arange(0, math.ceil(height / d) * r, r) % width)
    for r, d in slopes
)
day_03_p2 = math.prod(np.sum(tree_map[s]) for s in slope_index_generator)

In [ ]:
# noinspection PyTypeChecker
T = TypeVar("T", bound="Passport")

@dataclass
class Passport:
    byr: str = ""
    iyr: str = ""
    eyr: str = ""
    hgt: str = ""
    hcl: str = ""
    ecl: str = ""
    pid: str = ""
    cid: str = ""

    @classmethod
    def from_string(cls: Type[T], info: str) -> T:
        # I guess this an IDE bug as creating a dict from a generator should be valid
        # noinspection PyTypeChecker
        return Passport(
            **dict(p.split(":") for p in info.strip().replace("\n", " ").split())
        )

    def is_valid(self, excludes: list[str] = ("cid",), strict: bool = False) -> bool:
        def check_year_bounds(year: str, min_bound: int, max_bound: int) -> bool:
            return year.isdigit() and min_bound <= int(year) <= max_bound

        # noinspection PyShadowingNames
        def check_height(height: str) -> bool:
            match = re.match(r"^(\d{2,3})(cm|in)$", height)
            if match:
                value, unit = match.groups()
                value = int(value)
                return 150 <= value <= 193 if unit == "cm" else 59 <= value <= 76
            return False

        complete = all(v != "" for k, v in asdict(self).items() if k not in excludes)
        if strict:
            return (
                complete
                and check_year_bounds(self.byr, 1920, 2002)
                and check_year_bounds(self.iyr, 2010, 2020)
                and check_year_bounds(self.eyr, 2020, 2030)
                and check_height(self.hgt)
                and bool(re.match(r"^#[0-9a-f]{6}$", self.hcl))
                and self.ecl in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
                and bool(re.match(r"^\d{9}$", self.pid))
            )

        return complete

In [ ]:
%%timeit
with open("day_04_input.txt") as input_data:
    passport_infos = input_data.read().strip().split("\n\n")

day_04_p1 = sum(Passport.from_string(s).is_valid() for s in passport_infos)

In [ ]:
%%timeit
with open("day_04_input.txt") as input_data:
    passport_infos = input_data.read().strip().split("\n\n")

day_04_p2 = sum(Passport.from_string(s).is_valid(strict=True) for s in passport_infos)

In [ ]:
def seat_id(input_string: str) -> int:
    return int(input_string.translate(str.maketrans("FLBR", "0011")), 2)

In [ ]:
%%timeit
with open("day_05_input.txt") as input_data:
    seat_ids = sorted(map(seat_id, input_data))
day_05_p1 = seat_ids[-1]

In [ ]:
%%timeit
with open("day_05_input.txt") as input_data:
    seat_ids = sorted(map(seat_id, input_data))
day_05_p2 = seat_ids[np.nonzero(np.diff(seat_ids) == 2)[0][0]] + 1

In [ ]:
%%timeit
with open("day_06_input.txt") as input_data:
    groups = [{q for q in g.replace("\n", "")} for g in input_data.read().split("\n\n")]
day_06_p1 = sum(len(x) for x in groups)

In [ ]:
%%timeit
with open("day_06_input.txt") as input_data:
    groups = [[set(q) for q in g.split("\n")] for g in input_data.read().split("\n\n")]
day_06_p2 = sum(len(set.intersection(*[y for y in x])) for x in groups)

In [3]:
bag_regex = r"^(\w+ \w+)"
contained_regex = r"(\d+) (\w+ \w+)"


def parse_rule(input_string: str) -> tuple:
    bag_match = re.match(bag_regex, input_string).group(1)
    contained_match = [m.groups() for m in re.finditer(contained_regex, input_string)]
    return bag_match, [(int(m[0]), m[1]) for m in contained_match]

In [ ]:
%%timeit
with open("day_07_input.txt") as input_data:
    bag_map = dict(map(parse_rule, input_data))

@lru_cache()
def contains_bag(query_bag: str, bag_to_check: str) -> bool:
    contained_bags = {b for _, b in bag_map[bag_to_check]}
    if not contained_bags:
        return False
    if query_bag in contained_bags:
        return True
    return any(contains_bag(query_bag, b) for b in contained_bags)


day_07_p1 = sum(contains_bag("shiny gold", s) for s in bag_map)

In [ ]:
%%timeit
with open("day_07_input.txt") as input_data:
    bag_map = dict(map(parse_rule, input_data))

lookup_dict = defaultdict(set)
for bag, contained_bags in bag_map.items():
    for _, contained_bag in contained_bags:
        lookup_dict[contained_bag].add(bag)

bags_containing_shiny_gold = lookup_dict["shiny gold"]
bags_to_process = bags_containing_shiny_gold.copy()
while bags_to_process:
    new_bags = lookup_dict[bags_to_process.pop()]
    bags_containing_shiny_gold |= new_bags
    bags_to_process |= new_bags

day_07_p1 = len(bags_containing_shiny_gold)

In [4]:
%%timeit
with open("day_07_input.txt") as input_data:
    bag_map = dict(map(parse_rule, input_data))

def get_sub_bag_count(count: int, bag: str) -> int:
    if not bag_map[bag]:
        return count
    return count + count * sum(get_sub_bag_count(c, b) for c, b in bag_map[bag])


day_07_p1 = get_sub_bag_count(1, 'shiny gold') - 1
s

NameError: name 're' is not defined